In [1]:
import math
import itertools
import numpy as np

import hoomd
import gsd.hoomd
import coxeter

#### particle vertices, faces, and insphere radius

In [2]:
compress = gsd.hoomd.open('./DATA/compress.gsd')[-1]
verts = compress.particles.type_shapes[0]['vertices']
particle = coxeter.shapes.ConvexPolyhedron(verts)
faces = particle.faces
sigma = 2*particle.insphere_from_center.radius

#### parameters

In [3]:
# integration timestep
dt=0.0005

# coupling
kT=1.0
tau=100*dt

# reduced pressure
pressure=12*kT/particle.volume
tauS=1000*dt
pressure

11.999982943782193

#### simulation

In [4]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=20)
sim.create_state_from_gsd(filename='./DATA/compress.gsd', frame=-1)

#### potential

In [5]:
nl = hoomd.md.nlist.Cell(buffer=0.4)

alj = hoomd.md.pair.aniso.ALJ(nl)
alj.r_cut[('A', 'A')] = 2*particle.circumsphere_from_center.radius + 0.15*sigma
alj.params[('A', 'A')] = dict(epsilon=0.1, 
                              sigma_i=sigma, 
                              sigma_j=sigma, 
                              alpha=0)
alj.shape['A'] = dict(vertices=verts, 
                      faces=faces, 
                      rounding_radii=0) # rounding_radii=0.15*(sigma/2)

/Users/lina492375qw1188/opt/miniconda3/envs/alchemy/lib/python3.9/site-packages/hoomd/md/pair/aniso.py:589: FutureWarning: 'mode' key is deprectated and will be removed in hoomd 4.0.
  warnings.warn(


#### thermodynamic ensemble

In [6]:
npt = hoomd.md.methods.NPT(filter=hoomd.filter.All(), 
                           kT=kT, 
                           tau=tau, 
                           S=pressure, 
                           tauS=tauS,
                           couple='xyz')

/Users/lina492375qw1188/opt/miniconda3/envs/alchemy/lib/python3.9/site-packages/hoomd/md/methods/methods.py:365: FutureWarning: NPT is deprecated and wil be removed in hoomd 4.0. In version 4.0, use the ConstantPressure method with the desired thermostat from hoomd.md.methods.thermostats.
  warnings.warn(


#### integrator

In [7]:
integrator = hoomd.md.Integrator(dt=dt, 
                                 methods=[npt],
                                 forces=[alj],
                                 integrate_rotational_dof=True)

#### writer

In [8]:
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())

logger = hoomd.logging.Logger()
logger.add(sim, quantities=['timestep', 'walltime'])
logger.add(alj, quantities=['type_shapes'])
logger.add(thermodynamic_properties)

gsd_writer = hoomd.write.GSD(filename='./DATA/trajectory.gsd',
                             trigger=hoomd.trigger.Periodic(1000),
                             log=logger,
                             mode='wb')

#### attaching operations

In [10]:
sim.operations.writers.append(gsd_writer)
sim.operations.integrator = integrator
sim.operations.computes.append(thermodynamic_properties)

#### run simulation

In [11]:
sim.run(0)
# npt.thermalize_thermostat_and_barostat_dof()
alj.type_shapes

[{'type': 'ConvexPolyhedron',
  'rounding_radius': 0.0545562,
  'vertices': [[0.727416, 0, 0.363708],
   [0.363708, 0.629961, 0.363708],
   [-0.363708, 0.629961, 0.363708],
   [-0.727416, 8.90827e-17, 0.363708],
   [-0.363708, -0.629961, 0.363708],
   [0.363708, -0.629961, 0.363708],
   [0.727416, 0, -0.363708],
   [0.363708, 0.629961, -0.363708],
   [-0.363708, 0.629961, -0.363708],
   [-0.727416, 8.90827e-17, -0.363708],
   [-0.363708, -0.629961, -0.363708],
   [0.363708, -0.629961, -0.363708]]}]

In [12]:
sim.run(1e5)

In [13]:
phi = particle.volume*sim.state.N_particles/sim.state.box.volume
print('Final packing fraction:', phi)

Final packing fraction: 0.45294842245812256
